# Applied Data Science Capstone

*This NoteBook is used for Applied Data Science Capstone*

In [1]:
import pandas as pd
import numpy as np
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl

In [2]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urllib.request.urlopen(url, context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')

In [7]:
# Retrieve all of the Table tags
tags = soup('table')
locationDict = dict()
for tag in tags:
    if(tag.get('class', None) == ['wikitable', 'sortable']):
        t_body = tag.contents[1]
        dfIndex=0
        for index,row in enumerate(t_body.contents):
            if(row!='\n' and index>0):
                locationDict[dfIndex]={"Postal Code":row.contents[1].contents[0].rstrip('\n'),"Borough":row.contents[3].contents[0].rstrip('\n'),"Neighborhood":row.contents[5].contents[0].rstrip('\n')}
                dfIndex+=1
finalDf = pd.DataFrame.from_dict(locationDict, orient='index')
finalDf.describe()

,Postal Code,Borough,Neighborhood
count,180,180,180
unique,180,11,100
top,M2T,Not assigned,Not assigned
freq,1,77,77


In [19]:
#Cleaning Data
finalDf.drop(finalDf[finalDf['Borough'] == 'Not assigned'].index, inplace = True)

mask = finalDf['Neighborhood'] == "Not assigned"
finalDf.loc[mask,'Neighborhood'] = finalDf.loc[mask,'Borough']

finalDf.describe()

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M6G,North York,Downsview
freq,1,24,4


In [20]:
finalDf.shape

(103, 3)